<a href="https://colab.research.google.com/github/RedBatProject/Text-Classification/blob/main/Training_LSTM_LM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import pandas as pd

In [ ]:
!pip install datasets;
import transformers
import datasets

In [6]:
from datasets import load_dataset

dataset = load_dataset("wikitext",'wikitext-2-v1')

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

In [8]:
dataset['train']

Dataset({
    features: ['text'],
    num_rows: 36718
})

In [11]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length",max_length=8, truncation=True)


tokenized_datasets = dataset['train'].map(tokenize_function, batched=True)

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

In [129]:
sent = ''
for ind,i in enumerate(dataset['train']):
    # print(len(i['text'].split()))
    if len(i['text'].split())>0:
        sent+= " [CLS] " +i['text'] +' [SEP] '
        if ind == 200:
            break


In [130]:
len(sent.split())

10411

In [131]:
sent2 = tokenizer(sent,return_tensors='pt')['input_ids'][0,1:-1]

In [144]:
sent2.shape

torch.Size([12611])

In [207]:
bs = 32
x = torch.zeros(sent2.shape[0],9,dtype=torch.long)
# y = torch.zeros(sent2.shape[0]-8,dtype=torch.long)

In [208]:

for ind,i in enumerate(range(8,sent2.shape[0])):
    x[ind] = sent2[i-8:i+1]
    # y[ind] = sent2[i]

In [209]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(x, batch_size=32)

In [213]:
tokenizer.vocab_size

30000

In [214]:
embed = torch.randn(tokenizer.vocab_size,16)

In [215]:
embed

tensor([[ 0.4015,  1.0807,  0.8449,  ...,  0.3188,  0.9762, -2.1024],
        [ 0.0972,  0.3875,  0.8655,  ...,  0.1662, -2.2910,  0.3512],
        [-0.7299, -0.5123, -1.3024,  ...,  0.8916, -1.1639, -0.5081],
        ...,
        [-0.6831, -1.8320, -2.8514,  ..., -1.2181, -0.1573,  0.4660],
        [-0.0373,  0.4373,  0.4066,  ..., -0.9929, -0.0561, -0.3739],
        [ 0.5105, -0.4371, -0.6680,  ..., -0.3069,  0.8927, -0.8867]])

In [217]:
# embed[x].shape

torch.Size([12611, 9, 16])

In [212]:
for i in train_dataloader:
    x,y = i[:,0:8],i[:,8]
    break

tensor([[    2,   800,  3347,  3329,  2548, 13282,  1867,   800],
        [  800,  3347,  3329,  2548, 13282,  1867,   800,     3],
        [ 3347,  3329,  2548, 13282,  1867,   800,     3,     2],
        [ 3329,  2548, 13282,  1867,   800,     3,     2,  8252],
        [ 2548, 13282,  1867,   800,     3,     2,  8252,  1636],
        [13282,  1867,   800,     3,     2,  8252,  1636,    90],
        [ 1867,   800,     3,     2,  8252,  1636,    90,  3347],
        [  800,     3,     2,  8252,  1636,    90,  3347,  3329],
        [    3,     2,  8252,  1636,    90,  3347,  3329,  2548],
        [    2,  8252,  1636,    90,  3347,  3329,  2548,   203],
        [ 8252,  1636,    90,  3347,  3329,  2548,   203,    13],
        [ 1636,    90,  3347,  3329,  2548,   203,    13,    45],
        [   90,  3347,  3329,  2548,   203,    13,    45,     1],
        [ 3347,  3329,  2548,   203,    13,    45,     1, 13282],
        [ 3329,  2548,   203,    13,    45,     1, 13282,    13],
        [ 